In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
reset_seeds(SEED)

In [ ]:
train = pd.read_csv('train_final.csv')
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        644 non-null    object
 1   category  644 non-null    object
 2   질문_1      644 non-null    object
 3   질문_2      644 non-null    object
 4   질문_3      644 non-null    object
 5   질문_4      644 non-null    object
 6   답변_1      644 non-null    object
 7   답변_2      644 non-null    object
 8   답변_3      644 non-null    object
 9   답변_4      644 non-null    object
 10  답변_5      644 non-null    object
 11  답변_6      644 non-null    object
 12  답변_7      644 non-null    object
 13  답변_8      644 non-null    object
 14  답변_9      644 non-null    object
 15  답변_10     644 non-null    object
dtypes: object(16)
memory usage: 80.6+ KB


In [ ]:
category_lst = []
question_lst = []
answer_lst = []
for i in range(1, 5):
    for j in range(1, 11):
        for n in range(train.shape[0]):
            category = train['category'][n]
            question = train[f'질문_{i}'][n]
            answer = train[f'답변_{j}'][n]

            category_lst.append(category)
            question_lst.append(question)
            answer_lst.append(answer)

qna_df = pd.DataFrame({'category':category_lst,
                       'question':question_lst,
                       'answer':answer_lst})
qna_df

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,건축구조,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...
25755,타 마감하자,벽장 부위 결로의 주된 원인은 무엇인가요?,벽장 부분에 결로가 생기는 가장 큰 이유는 충분한 환기가 되지 않는 환경과 과도한 ...
25756,타 마감하자,벽체 결로가 AD나 PD에 영향을 주는 가장 큰 이유는 무엇인가요?,"벽체에 발생하는 결로의 원인 중 하나는, 벽면이 내부보다 외부 공기에 노출되어서 냉..."
25757,타 마감하자,외벽 모서리에 생기는 결로를 방지할 수 있는 방법은 있을까요?,외벽 모서리에 결로가 발생하는 주요 이유는 높은 온도차입니다. 외벽 모서리는 열이 ...
25758,타 마감하자,창호 결로를 제거하는 가장 효과적인 방법은 무엇일까요?,KS규격에 따른 프레임 사용과 열전달 및 결로시험 수행은 창호 결로 방지를 위한 필...


In [ ]:
from itertools import permutations

reset_seeds(SEED)

# 질문과 답변을 무작위로 2개 선택하여 연결어를 통해 하나의 문장으로 생성
num = train.shape[0]
final_num = 80000

idx_lst = list(range(num))
idx_lsts = list(permutations(idx_lst, 2))
sample_list = random.sample(idx_lsts, final_num-qna_df.shape[0])

aug_category = []
aug_questions = []
aug_answers = []
for i, j in sample_list:

    # 연결어 랜덤 선택
    connect_words = [' 그리고 ', ' 또한, ', ' 또 ']
    selected_connect = random.choice(connect_words)

    if qna_df['category'][i][0] == qna_df['category'][j][0]:
        aug_c = qna_df['category'][i]
    else:
        aug_c = qna_df['category'][i]+ qna_df['category'][j]
    aug_q = qna_df['question'][i] + selected_connect + qna_df['question'][j]
    aug_a = qna_df['answer'][i] + selected_connect  + qna_df['answer'][j]

    aug_category.append(aug_c)
    aug_questions.append(aug_q)
    aug_answers.append(aug_a)

aug_df_1 = pd.DataFrame({'category':aug_category,
                       'question':aug_questions,
                       'answer':aug_answers})
aug_df_1

,category,question,answer
0,인테리어마감재,어떻게 테라스를 활용하여 야외 공간을 아늑하게 꾸밀 수 있을까요? 또 비드법 보온판...,"테라스를 아늑하게 만들려면 야외 가구, 식물, 그리고 외부 조명을 활용하세요. 또 ..."
1,마감재시공,"송판 노출콘크리트가 뭐야? 또한, 초배작업은 필수야?",송판 노출콘크리트란 나무무늬의 질감이 나는 콘크리트 외장재 마감을 의미합니다. 또한...
2,건축구조마감재,"경량철골구조가 뭐야? 또한, 아크릴 계열 페인트는 어디에 사용해?","경량철골구조는 약 1.5~3mm 두께의 철골을 사용하며 각각 파이프, C형 찬넬, ..."
3,인테리어시공,"어떻게 아이 방을 아기자기하게 꾸밀 수 있을까요? 또한, 방청페인트 시공방법이 뭐야?","아이 방을 아기자기하게 만들려면 패턴 풍성한 커튼, 아기자기한 벽지, 그리고 재미있..."
4,마감재,"포세린 타일의 장점이 뭐야? 또한, 점토벽돌의 단점이 뭐야?",포세린 타일은 밀도와 강도가 뛰어나고 변형이 적고 반영구적이라는 장점이 있습니다. ...
...,...,...,...
54235,시공타 마감하자,왜 초배지만 남은 벽에는 페인트를 못 칠해? 그리고 지하 공간 결로의 원인이 뭐야?,초배지는 매우 얇기 얇아 페인트를 바로 칠하면 표면이 쭈글쭈글하게 울기에 초배지 위...
54236,마감재시공,"차음재 중 이중벽은 뭐가 있어? 또한, 올퍼티 마감이 뭐야?",차음재 중 이중벽은 내부의 공기층을 형성한 중공이중벽과 내부에 다공질 흡음재를 설치...
54237,마감하자시공,도배지에 울음이 생겼어 그리고 액체방수공사가 뭐야?,도배지 울음이란 도배지가 평평하지 않고 국소적으로 볼록하게 튀어나오는 현상을 의미합...
54238,인테리어마감하자,인테리어에서 텍스처를 활용하는 방법은 어떤 게 있나요? 또 몰딩수정은 뭐야?,"다양한 소재와 질감의 가구나 소품, 벽면에 텍스처를 활용하여 공간에 깊이와 풍부한 ..."


In [ ]:
# 질문이 뭐야, 또는 무엇으로 끝나는 질문 선택
idx_lst = []
for idx, row in train.iterrows():
    if row['질문_1'].find('뭐야') != -1 or row['질문_1'].find('무엇') != -1:
        idx_lst.append(idx)

tmp = train.loc[idx_lst]
tmp.reset_index(inplace=True)

# 무작위로 섞은 후에 tmp_1은 무엇인지, tmp_2는 무엇이며, tmp_3은 무엇이고, 로 끝나는 문장으로 변경
tmp = tmp.sample(frac=1, random_state=SEED, ignore_index=True)

tmp_1 = tmp.iloc[:112].copy().reset_index(drop=True)
tmp_2 = tmp.iloc[112:112*2].copy().reset_index(drop=True)
tmp_3 = tmp.iloc[112*2:].copy().reset_index(drop=True)

for index, text in enumerate(tmp_1['질문_1']):
    word = text.split()[-1]
    text = text.replace(f'{word}', '무엇인지,')
    tmp_1.loc[index, '질문_1'] = text

for index, text in enumerate(tmp_2['질문_1']):
    word = text.split()[-1]
    text = text.replace(f'{word}', '무엇이며,')
    tmp_2.loc[index, '질문_1'] = text

for index, text in enumerate(tmp_3['질문_1']):
    word = text.split()[-1]
    text = text.replace(f'{word}', '무엇이고,')
    tmp_3.loc[index, '질문_1'] = text

tmp = pd.concat([tmp_1, tmp_2, tmp_3]).reset_index(drop=True)

In [ ]:
# 무엇인지, 무엇이며, 무엇이고로 끝나는 질문이 앞에 오게 한 후 다른 문장과 연결

train_1 = train.loc[:, ['category', '질문_1', '답변_1']].copy()
train_2 = tmp.loc[:, ['index', '질문_1']].copy()

for _, row in train_2.iterrows():
    train_1.loc[row['index'], '질문_1'] = row['질문_1']

change_lst = list(train_2['index'])

reset_seeds(SEED)

aug_category = []
aug_questions = []
aug_answers = []
for i in tqdm(change_lst):
    for j in range(train.shape[0]):
        if i == j: # 같은 질문이 선택되는 것을 방지
            continue
        # 연결어 랜덤 선택
        connect_words = [' 그리고 ', ' 또한, ', ' 또 ']
        selected_connect = random.choice(connect_words)

        if train_1['category'][i][0] == train['category'][j][0]:
            aug_c = train_1['category'][i]
        else:
            aug_c = train_1['category'][i] + train['category'][j]

        # 인지로 끝나는 문장은 연결어를 사용하여 연결
        if train_1['질문_1'][i].endswith("인지,"):
            aug_q = train_1['질문_1'][i] + selected_connect + train['질문_1'][j]
        else:
            aug_q = train_1['질문_1'][i] + " " + train['질문_1'][j]

        aug_a = train_1['답변_1'][i] + selected_connect  + train['답변_1'][j]

        aug_category.append(aug_c)
        aug_questions.append(aug_q)
        aug_answers.append(aug_a)

aug_df_2 = pd.DataFrame({'category':aug_category,
                       'question':aug_questions,
                       'answer':aug_answers})
# 무작위로 섞은 후에 4만개만 사용.
aug_df_2 = aug_df_2.sample(frac=1, random_state=SEED, ignore_index=True).iloc[:40000]

  0%|          | 0/335 [00:00<?, ?it/s]

In [ ]:
train_df = pd.concat([qna_df, aug_df_1, aug_df_2]).reset_index(drop=True)
train_df.shape

(120000, 3)

In [ ]:
train_df.to_csv("train_120000_final.csv", index=False, encoding='utf-8-sig')